# SSD300 Object Detection - Complete Training & Evaluation Pipeline

This notebook provides a comprehensive implementation for training and evaluating SSD300 on Pascal VOC 2012 dataset with enhanced visualizations, logging, and model comparison capabilities.

## Features:
- Complete training pipeline with real-time logging
- Comprehensive evaluation with COCO metrics
- Enhanced visualizations and performance analysis
- Model comparison support using predefined test images
- Detailed logging and progress tracking
- Professional reporting and result export

## 1. Environment Setup and Configuration

In [ ]:
import os
import sys
import json
import time
import logging
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm
import cv2
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
import torchvision.transforms as T
from torchvision.models.detection import ssd300_vgg16, SSD300_VGG16_Weights
from torchvision.models.detection.ssd import SSDClassificationHead
from torchvision.ops import nms

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# Add project root to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
from SSD300.src.coco_voc import COCODetectionDataset

# Configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using device: {DEVICE}")
print(f"📅 Session started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Pascal VOC classes
VOC_CLASSES = [
    "aeroplane", "bicycle", "bird", "boat", "bottle",
    "bus", "car", "cat", "chair", "cow",
    "diningtable", "dog", "horse", "motorbike", "person",
    "pottedplant", "sheep", "sofa", "train", "tvmonitor"
]

# Create output directories
os.makedirs("outputs", exist_ok=True)
os.makedirs("outputs/logs", exist_ok=True)
os.makedirs("outputs/visualizations", exist_ok=True)
os.makedirs("outputs/models", exist_ok=True)
os.makedirs("outputs/predictions", exist_ok=True)

## 2. Enhanced Logging Setup

In [ ]:
# Setup comprehensive logging
def setup_logging():
    """Setup logging configuration for training session."""
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    log_file = f"outputs/logs/ssd300_training_{timestamp}.log"
    
    # Configure logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )
    
    logger = logging.getLogger(__name__)
    logger.info(f"🚀 SSD300 Training Session Started")
    logger.info(f"📝 Log file: {log_file}")
    logger.info(f"🖥️  Device: {DEVICE}")
    
    return logger, log_file

logger, log_file = setup_logging()
print(f"✅ Logging setup complete: {log_file}")

## 3. Load Comparison Images Configuration

In [ ]:
# Load comparison images configuration
COMPARISON_CONFIG_PATH = "../../comparison_images.json"

try:
    with open(COMPARISON_CONFIG_PATH, 'r') as f:
        comparison_config = json.load(f)
    
    comparison_images = comparison_config['images']
    logger.info(f"📸 Loaded {len(comparison_images)} comparison images for model evaluation")
    
    # Display sample images info
    print("🔍 Sample comparison images:")
    for i, img_info in enumerate(comparison_images[:5]):
        print(f"  {i+1}. {img_info['filename']} - {img_info['description']}")
    print(f"  ... and {len(comparison_images)-5} more images")
    
except FileNotFoundError:
    logger.warning(f"⚠️  Comparison config not found at {COMPARISON_CONFIG_PATH}")
    comparison_images = []

# Training configuration
CONFIG = {
    'batch_size': 4,
    'epochs': 5,
    'learning_rate': 1e-4,
    'num_classes': 21,
    'image_size': 300,
    'confidence_threshold': 0.3,
    'nms_threshold': 0.45
}

logger.info(f"⚙️  Training configuration: {CONFIG}")

## 4. Data Preparation with Progress Tracking

In [ ]:
# Dataset paths (update according to your setup)
DATA_CONFIG = {
    'voc_data_dir': "../../data/VOCdevkit/VOC2012",
    'images_dir': "../../data/VOCdevkit/VOC2012/JPEGImages",
    'annotations_dir': "../../data/VOCdevkit/VOC2012/Annotations",
    'trainval_ids': "../../data/VOCdevkit/VOC2012/ImageSets/Main/trainval.txt",
    'coco_json_path': "../../data/voc2012_coco.json",
    'labels_file': "../../data/labels.txt"
}

# Create labels file
logger.info("📝 Creating Pascal VOC labels file...")
os.makedirs(os.path.dirname(DATA_CONFIG['labels_file']), exist_ok=True)
with open(DATA_CONFIG['labels_file'], 'w') as f:
    for cls in VOC_CLASSES:
        f.write(f"{cls}\n")

# Check if COCO conversion is needed
if not os.path.exists(DATA_CONFIG['coco_json_path']):
    logger.info("🔄 Converting VOC annotations to COCO format...")
    conversion_cmd = f"""
    python ../src/voc2coco.py \
      --ann_dir {DATA_CONFIG['annotations_dir']} \
      --ann_ids {DATA_CONFIG['trainval_ids']} \
      --labels {DATA_CONFIG['labels_file']} \
      --output {DATA_CONFIG['coco_json_path']} \
      --ext xml \
      --extract_num_from_imgid
    """
    
    result = os.system(conversion_cmd.strip())
    if result == 0:
        logger.info(f"✅ VOC to COCO conversion completed: {DATA_CONFIG['coco_json_path']}")
    else:
        logger.error(f"❌ VOC to COCO conversion failed")
        raise RuntimeError("Data preparation failed")
else:
    logger.info(f"✅ COCO annotations already exist: {DATA_CONFIG['coco_json_path']}")

print(f"📊 Dataset configuration: {DATA_CONFIG}")

## 5. Enhanced Dataset Loading with Metrics

In [ ]:
def get_transform():
    """Get transforms for SSD300 input preprocessing."""
    return T.Compose([
        T.Resize((CONFIG['image_size'], CONFIG['image_size'])),
        T.ToTensor(),
        T.Normalize(mean=[0.48235, 0.45882, 0.40784], 
                   std=[0.229, 0.224, 0.225])
    ])

def collate_fn(batch):
    """Custom collate function for object detection."""
    return tuple(zip(*batch))

# Create dataset and analyze
logger.info("📊 Loading and analyzing dataset...")
dataset = COCODetectionDataset(
    DATA_CONFIG['coco_json_path'], 
    DATA_CONFIG['images_dir'], 
    transform=get_transform()
)

# Dataset statistics
coco_gt = COCO(DATA_CONFIG['coco_json_path'])
dataset_stats = {
    'total_images': len(dataset),
    'total_annotations': len(coco_gt.anns),
    'categories': len(coco_gt.cats),
    'avg_annotations_per_image': len(coco_gt.anns) / len(dataset)
}

logger.info(f"📈 Dataset Statistics: {dataset_stats}")

# Create dataloader
dataloader = DataLoader(
    dataset, 
    batch_size=CONFIG['batch_size'], 
    shuffle=True, 
    collate_fn=collate_fn,
    num_workers=2,
    pin_memory=True if DEVICE.type == 'cuda' else False
)

print(f"✅ Dataset ready: {len(dataset)} images, {len(dataloader)} batches")

## 6. Model Setup with Detailed Analysis

In [ ]:
# Initialize SSD300 model
logger.info("🧠 Initializing SSD300 model...")
model = ssd300_vgg16(weights=SSD300_VGG16_Weights.COCO_V1)

# Replace classification head for Pascal VOC
model.head.classification_head = SSDClassificationHead(
    in_channels=[512, 1024, 512, 256, 256, 256],
    num_anchors=[4, 6, 6, 6, 4, 4],
    num_classes=CONFIG['num_classes']
)

model.to(DEVICE)

# Model analysis
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
frozen_params = total_params - trainable_params

model_info = {
    'total_parameters': total_params,
    'trainable_parameters': trainable_params,
    'frozen_parameters': frozen_params,
    'model_size_mb': total_params * 4 / (1024 * 1024),  # Assuming float32
    'device': str(DEVICE)
}

logger.info(f"🔧 Model Info: {model_info}")

# Setup optimizer with logging
optimizer = Adam(model.parameters(), lr=CONFIG['learning_rate'])

print(f"✅ Model ready on {DEVICE}")
print(f"📊 Parameters: {total_params:,} total, {trainable_params:,} trainable")
print(f"💾 Estimated model size: {model_info['model_size_mb']:.1f} MB")

## 7. Enhanced Training Loop with Real-time Visualization

In [ ]:
# Training tracking variables
training_log = {
    'epoch_losses': [],
    'batch_losses': [],
    'learning_rates': [],
    'training_times': [],
    'start_time': time.time()
}

# Training function with enhanced logging
def train_model(model, dataloader, optimizer, num_epochs, device):
    """Enhanced training function with comprehensive logging."""
    model.train()
    
    logger.info(f"🚀 Starting training for {num_epochs} epochs")
    logger.info(f"📊 Batch size: {CONFIG['batch_size']}, Total batches: {len(dataloader)}")
    
    for epoch in range(num_epochs):
        epoch_start = time.time()
        epoch_loss = 0.0
        
        # Progress bar for epoch
        progress_bar = tqdm(
            dataloader, 
            desc=f"Epoch {epoch+1}/{num_epochs}",
            leave=True
        )
        
        for batch_idx, (images, targets) in enumerate(progress_bar):
            # Move data to device
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            # Forward pass
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            
            # Backward pass
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
            
            # Update metrics
            batch_loss = losses.item()
            epoch_loss += batch_loss
            training_log['batch_losses'].append(batch_loss)
            
            # Update progress bar
            current_avg = epoch_loss / (batch_idx + 1)
            progress_bar.set_postfix({
                'batch_loss': f'{batch_loss:.4f}',
                'avg_loss': f'{current_avg:.4f}',
                'lr': f'{optimizer.param_groups[0]["lr"]:.2e}'
            })
            
            # Log every 100 batches
            if (batch_idx + 1) % 100 == 0:
                logger.info(
                    f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx+1}/{len(dataloader)}: "
                    f"Loss = {batch_loss:.4f}, Avg = {current_avg:.4f}"
                )
        
        # Epoch summary
        epoch_time = time.time() - epoch_start
        avg_epoch_loss = epoch_loss / len(dataloader)
        
        training_log['epoch_losses'].append(avg_epoch_loss)
        training_log['training_times'].append(epoch_time)
        training_log['learning_rates'].append(optimizer.param_groups[0]['lr'])
        
        logger.info(
            f"✅ Epoch {epoch+1}/{num_epochs} completed in {epoch_time:.2f}s: "
            f"Avg Loss = {avg_epoch_loss:.4f}"
        )
        
        # Save checkpoint every epoch
        checkpoint_path = f"outputs/models/ssd300_epoch_{epoch+1}.pth"
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_epoch_loss,
            'config': CONFIG
        }, checkpoint_path)
        
        logger.info(f"💾 Checkpoint saved: {checkpoint_path}")
    
    total_training_time = time.time() - training_log['start_time']
    training_log['total_time'] = total_training_time
    
    logger.info(f"🏁 Training completed in {total_training_time:.2f}s")
    return training_log

# Start training
print("🚀 Starting model training...")
training_results = train_model(
    model, dataloader, optimizer, CONFIG['epochs'], DEVICE
)

# Save final model
final_model_path = "outputs/models/ssd300_final.pth"
torch.save(model.state_dict(), final_model_path)
logger.info(f"💾 Final model saved: {final_model_path}")

## 8. Training Visualization and Analysis

In [ ]:
# Create comprehensive training visualizations
def create_training_visualizations(training_log):
    """Create detailed training visualizations."""
    fig = plt.figure(figsize=(20, 12))
    
    # 1. Epoch Loss Curve
    ax1 = plt.subplot(2, 3, 1)
    epochs = range(1, len(training_log['epoch_losses']) + 1)
    plt.plot(epochs, training_log['epoch_losses'], 'b-o', linewidth=2, markersize=8)
    plt.title('Training Loss per Epoch', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Average Loss')
    plt.grid(True, alpha=0.3)
    
    # Add loss values as annotations
    for i, loss in enumerate(training_log['epoch_losses']):
        plt.annotate(f'{loss:.3f}', (i+1, loss), textcoords="offset points", 
                    xytext=(0,10), ha='center', fontsize=10)
    
    # 2. Batch Loss Trend (smoothed)
    ax2 = plt.subplot(2, 3, 2)
    batch_losses = training_log['batch_losses']
    # Moving average for smoothing
    window = max(1, len(batch_losses) // 100)
    smoothed_losses = np.convolve(batch_losses, np.ones(window)/window, mode='valid')
    
    plt.plot(smoothed_losses, 'r-', alpha=0.7, linewidth=1)
    plt.title(f'Batch Losses (Smoothed, window={window})', fontsize=14, fontweight='bold')
    plt.xlabel('Batch')
    plt.ylabel('Loss')
    plt.grid(True, alpha=0.3)
    
    # 3. Training Time per Epoch
    ax3 = plt.subplot(2, 3, 3)
    plt.bar(epochs, training_log['training_times'], color='green', alpha=0.7)
    plt.title('Training Time per Epoch', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Time (seconds)')
    plt.grid(True, alpha=0.3, axis='y')
    
    # Add time values as annotations
    for i, time_val in enumerate(training_log['training_times']):
        plt.text(i+1, time_val + max(training_log['training_times'])*0.01, 
                f'{time_val:.1f}s', ha='center', va='bottom', fontsize=10)
    
    # 4. Loss Improvement
    ax4 = plt.subplot(2, 3, 4)
    if len(training_log['epoch_losses']) > 1:
        improvements = []
        for i in range(1, len(training_log['epoch_losses'])):
            improvement = training_log['epoch_losses'][i-1] - training_log['epoch_losses'][i]
            improvements.append(improvement)
        
        plt.bar(range(2, len(training_log['epoch_losses']) + 1), improvements, 
               color='orange', alpha=0.7)
        plt.title('Loss Improvement per Epoch', fontsize=14, fontweight='bold')
        plt.xlabel('Epoch')
        plt.ylabel('Loss Reduction')
        plt.grid(True, alpha=0.3, axis='y')
    
    # 5. Learning Rate Schedule
    ax5 = plt.subplot(2, 3, 5)
    plt.plot(epochs, training_log['learning_rates'], 'purple', marker='s', linewidth=2)
    plt.title('Learning Rate Schedule', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Learning Rate')
    plt.yscale('log')
    plt.grid(True, alpha=0.3)
    
    # 6. Training Summary Statistics
    ax6 = plt.subplot(2, 3, 6)
    ax6.axis('off')
    
    # Calculate statistics
    initial_loss = training_log['epoch_losses'][0]
    final_loss = training_log['epoch_losses'][-1]
    loss_reduction = ((initial_loss - final_loss) / initial_loss) * 100
    avg_epoch_time = np.mean(training_log['training_times'])
    total_time = training_log['total_time']
    
    summary_text = f"""
    Training Summary
    ================
    
    📊 Loss Statistics:
    • Initial Loss: {initial_loss:.4f}
    • Final Loss: {final_loss:.4f}
    • Improvement: {loss_reduction:.1f}%
    
    ⏱️ Time Statistics:
    • Total Time: {total_time:.0f}s ({total_time/60:.1f}m)
    • Avg per Epoch: {avg_epoch_time:.1f}s
    
    🔧 Configuration:
    • Epochs: {CONFIG['epochs']}
    • Batch Size: {CONFIG['batch_size']}
    • Learning Rate: {CONFIG['learning_rate']}
    • Device: {DEVICE}
    """
    
    ax6.text(0.1, 0.9, summary_text, transform=ax6.transAxes, fontsize=11,
            verticalalignment='top', fontfamily='monospace',
            bbox=dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.8))
    
    plt.tight_layout()
    plt.suptitle('SSD300 Training Analysis Dashboard', fontsize=16, fontweight='bold', y=0.98)
    
    # Save visualization
    viz_path = 'outputs/visualizations/training_analysis.png'
    plt.savefig(viz_path, dpi=300, bbox_inches='tight')
    logger.info(f"📊 Training visualization saved: {viz_path}")
    
    plt.show()

# Create visualizations
create_training_visualizations(training_results)

# Save training log
log_path = 'outputs/logs/training_log.json'
with open(log_path, 'w') as f:
    json.dump(training_results, f, indent=2)
logger.info(f"📝 Training log saved: {log_path}")

## 9. Comprehensive Model Evaluation

In [ ]:
# Enhanced evaluation function
def evaluate_model(model, dataset, device, save_predictions=True):
    """Comprehensive model evaluation with detailed metrics."""
    logger.info("🔍 Starting comprehensive model evaluation...")
    
    model.eval()
    eval_dataloader = DataLoader(dataset, batch_size=CONFIG['batch_size'], 
                                shuffle=False, collate_fn=collate_fn)
    
    # Load ground truth COCO
    coco_gt = COCO(DATA_CONFIG['coco_json_path'])
    if 'info' not in coco_gt.dataset:
        coco_gt.dataset['info'] = {"description": "VOC to COCO converted dataset"}
    
    all_predictions = []
    inference_times = []
    
    logger.info(f"📊 Running inference on {len(dataset)} images...")
    
    with torch.no_grad():
        for batch_idx, (images, targets) in enumerate(tqdm(eval_dataloader, desc="Evaluating")):
            batch_start = time.time()
            
            images = [img.to(device) for img in images]
            outputs = model(images)
            
            batch_time = time.time() - batch_start
            inference_times.append(batch_time / len(images))  # Per image time
            
            for output, target in zip(outputs, targets):
                img_id = target['image_id'].item()
                
                # Apply confidence filtering
                conf_mask = output['scores'] > CONFIG['confidence_threshold']
                if conf_mask.sum() == 0:
                    continue
                
                boxes = output['boxes'][conf_mask]
                scores = output['scores'][conf_mask]
                labels = output['labels'][conf_mask]
                
                # Apply NMS
                keep = nms(boxes, scores, CONFIG['nms_threshold'])
                
                for i in keep:
                    box = boxes[i].cpu().tolist()
                    x1, y1, x2, y2 = box
                    all_predictions.append({
                        "image_id": img_id,
                        "category_id": int(labels[i]) + 1,
                        "bbox": [x1, y1, x2 - x1, y2 - y1],
                        "score": float(scores[i])
                    })
    
    # Calculate inference statistics
    avg_inference_time = np.mean(inference_times)
    fps = 1.0 / avg_inference_time
    
    logger.info(f"⚡ Inference speed: {fps:.2f} FPS ({avg_inference_time*1000:.1f} ms/image)")
    logger.info(f"📊 Generated {len(all_predictions)} predictions")
    
    # Save predictions
    if save_predictions:
        results_path = "outputs/predictions/ssd300_predictions.json"
        with open(results_path, 'w') as f:
            json.dump(all_predictions, f)
        logger.info(f"💾 Predictions saved: {results_path}")
    
    # Run COCO evaluation
    logger.info("📈 Computing COCO evaluation metrics...")
    
    if len(all_predictions) > 0:
        coco_dt = coco_gt.loadRes(all_predictions)
        coco_eval = COCOeval(coco_gt, coco_dt, iouType='bbox')
        
        # Filter to images with predictions
        img_ids_with_preds = sorted({pred['image_id'] for pred in all_predictions})
        coco_eval.params.imgIds = img_ids_with_preds
        
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()
        
        # Extract metrics
        metrics_keys = [
            "AP_0.50:0.95", "AP_0.50", "AP_0.75",
            "AP_small", "AP_medium", "AP_large",
            "AR_1", "AR_10", "AR_100",
            "AR_small", "AR_medium", "AR_large"
        ]
        
        evaluation_results = dict(zip(metrics_keys, coco_eval.stats.tolist()))
        evaluation_results['inference_fps'] = fps
        evaluation_results['avg_inference_time_ms'] = avg_inference_time * 1000
        evaluation_results['total_predictions'] = len(all_predictions)
        
        # Per-class AP
        precision = coco_eval.eval['precision']
        cat_ids = coco_eval.params.catIds
        cat_names = {cat['id']: cat['name'] for cat in coco_gt.loadCats(cat_ids)}
        
        per_class_ap = {}
        for i, cat_id in enumerate(cat_ids):
            cat_precision = precision[:, :, i, 0, 2]
            ap = np.mean(cat_precision[cat_precision > -1])
            per_class_ap[cat_names[cat_id]] = float(ap)
        
        evaluation_results['per_class_AP'] = per_class_ap
        
        return evaluation_results, coco_eval
    else:
        logger.warning("⚠️  No predictions generated - check confidence threshold")
        return None, None

# Run evaluation
eval_results, coco_evaluator = evaluate_model(model, dataset, DEVICE)

## 10. Comparison Image Testing and Visualization

In [ ]:
def test_comparison_images(model, comparison_images, images_dir, device):
    """Test model on comparison images and create visualizations."""
    logger.info(f"🎯 Testing model on {len(comparison_images)} comparison images...")
    
    model.eval()
    comparison_results = []
    
    # Preprocessing transform (without normalization for visualization)
    preprocess = T.Compose([
        T.Resize((CONFIG['image_size'], CONFIG['image_size'])),
        T.ToTensor()
    ])
    
    # Model input transform (with normalization)
    model_transform = T.Compose([
        T.Resize((CONFIG['image_size'], CONFIG['image_size'])),
        T.ToTensor(),
        T.Normalize(mean=[0.48235, 0.45882, 0.40784], 
                   std=[0.229, 0.224, 0.225])
    ])
    
    fig, axes = plt.subplots(4, 5, figsize=(25, 20))
    axes = axes.flatten()
    
    with torch.no_grad():
        for i, img_info in enumerate(comparison_images[:20]):  # Process first 20 images
            img_path = os.path.join(images_dir, img_info['filename'])
            
            if not os.path.exists(img_path):
                logger.warning(f"⚠️  Image not found: {img_path}")
                continue
            
            # Load and process image
            original_img = Image.open(img_path).convert('RGB')
            img_for_viz = preprocess(original_img)
            img_for_model = model_transform(original_img).unsqueeze(0).to(device)
            
            # Run inference
            start_time = time.time()
            prediction = model(img_for_model)[0]
            inference_time = time.time() - start_time
            
            # Filter predictions
            conf_mask = prediction['scores'] > CONFIG['confidence_threshold']
            
            if conf_mask.sum() > 0:
                boxes = prediction['boxes'][conf_mask]
                scores = prediction['scores'][conf_mask]
                labels = prediction['labels'][conf_mask]
                
                # Apply NMS
                keep = nms(boxes, scores, CONFIG['nms_threshold'])
                boxes = boxes[keep].cpu()
                scores = scores[keep].cpu()
                labels = labels[keep].cpu()
            else:
                boxes = torch.empty(0, 4)
                scores = torch.empty(0)
                labels = torch.empty(0)
            
            # Store results
            comparison_results.append({
                'image_info': img_info,
                'inference_time_ms': inference_time * 1000,
                'detections': len(boxes),
                'boxes': boxes.tolist(),
                'scores': scores.tolist(),
                'labels': labels.tolist()
            })
            
            # Visualize
            ax = axes[i]
            
            # Convert tensor to numpy for visualization
            img_np = img_for_viz.permute(1, 2, 0).numpy()
            ax.imshow(img_np)
            
            # Draw bounding boxes
            for box, score, label in zip(boxes, scores, labels):
                x1, y1, x2, y2 = box
                width, height = x2 - x1, y2 - y1
                
                # Create rectangle
                rect = patches.Rectangle((x1, y1), width, height, 
                                       linewidth=2, edgecolor='red', 
                                       facecolor='none')
                ax.add_patch(rect)
                
                # Add label
                class_name = VOC_CLASSES[int(label)]
                ax.text(x1, y1-5, f'{class_name}: {score:.2f}', 
                       color='red', fontsize=8, fontweight='bold',
                       bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.8))
            
            # Set title with info
            title = f"{img_info['image_id']}\n{len(boxes)} det, {inference_time*1000:.1f}ms"
            ax.set_title(title, fontsize=10)
            ax.axis('off')
    
    # Hide unused subplots
    for i in range(len(comparison_images), len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.suptitle('SSD300 Predictions on Comparison Images', fontsize=16, fontweight='bold')
    
    # Save visualization
    comparison_viz_path = 'outputs/visualizations/comparison_predictions.png'
    plt.savefig(comparison_viz_path, dpi=300, bbox_inches='tight')
    logger.info(f"🎨 Comparison visualization saved: {comparison_viz_path}")
    
    plt.show()
    
    return comparison_results

# Test on comparison images if available
if comparison_images:
    comparison_results = test_comparison_images(
        model, comparison_images, DATA_CONFIG['images_dir'], DEVICE
    )
    
    # Save comparison results
    comparison_path = 'outputs/predictions/ssd300_comparison_results.json'
    with open(comparison_path, 'w') as f:
        json.dump(comparison_results, f, indent=2)
    logger.info(f"💾 Comparison results saved: {comparison_path}")
else:
    logger.info("⏭️  Skipping comparison images (not available)")

## 11. Performance Analysis and Final Report

In [ ]:
# Create comprehensive performance report
def create_final_report(training_results, eval_results, model_info, comparison_results=None):
    """Generate comprehensive final report."""
    
    report = {
        'session_info': {
            'timestamp': datetime.now().isoformat(),
            'device': str(DEVICE),
            'pytorch_version': torch.__version__
        },
        'configuration': CONFIG,
        'dataset_stats': dataset_stats,
        'model_info': model_info,
        'training_results': {
            'final_loss': training_results['epoch_losses'][-1],
            'loss_reduction_percent': ((training_results['epoch_losses'][0] - training_results['epoch_losses'][-1]) / training_results['epoch_losses'][0]) * 100,
            'total_training_time_minutes': training_results['total_time'] / 60,
            'avg_epoch_time_minutes': np.mean(training_results['training_times']) / 60
        },
        'evaluation_results': eval_results if eval_results else {},
        'files_generated': {
            'final_model': 'outputs/models/ssd300_final.pth',
            'predictions': 'outputs/predictions/ssd300_predictions.json',
            'training_log': 'outputs/logs/training_log.json',
            'visualizations': [
                'outputs/visualizations/training_analysis.png',
                'outputs/visualizations/comparison_predictions.png'
            ]
        }
    }
    
    if comparison_results:
        # Analyze comparison results
        total_detections = sum(r['detections'] for r in comparison_results)
        avg_inference_time = np.mean([r['inference_time_ms'] for r in comparison_results])
        
        report['comparison_analysis'] = {
            'images_tested': len(comparison_results),
            'total_detections': total_detections,
            'avg_detections_per_image': total_detections / len(comparison_results),
            'avg_inference_time_ms': avg_inference_time
        }
    
    return report

# Generate final report
final_report = create_final_report(
    training_results, 
    eval_results, 
    model_info, 
    comparison_results if 'comparison_results' in locals() else None
)

# Save final report
report_path = 'outputs/ssd300_final_report.json'
with open(report_path, 'w') as f:
    json.dump(final_report, f, indent=2)

logger.info(f"📊 Final report saved: {report_path}")

# Print summary
print("\n" + "="*80)
print("🎉 SSD300 TRAINING & EVALUATION COMPLETED SUCCESSFULLY! 🎉")
print("="*80)

if eval_results:
    print(f"\n📈 KEY RESULTS:")
    print(f"  • mAP @ IoU=0.50:0.95: {eval_results['AP_0.50:0.95']:.3f}")
    print(f"  • mAP @ IoU=0.50:     {eval_results['AP_0.50']:.3f}")
    print(f"  • Inference Speed:    {eval_results['inference_fps']:.1f} FPS")
    print(f"  • Training Loss:      {training_results['epoch_losses'][0]:.3f} → {training_results['epoch_losses'][-1]:.3f}")
    print(f"  • Training Time:      {training_results['total_time']/60:.1f} minutes")

print(f"\n📁 OUTPUT FILES:")
print(f"  • Model:        outputs/models/ssd300_final.pth")
print(f"  • Predictions:  outputs/predictions/ssd300_predictions.json")
print(f"  • Report:       {report_path}")
print(f"  • Logs:         {log_file}")
print(f"  • Visualizations: outputs/visualizations/")

print(f"\n🏆 Ready for model comparison with Faster R-CNN and YOLOv5!")
print("="*80)

logger.info("✅ SSD300 pipeline completed successfully!")